In [1]:
!pip install pybullet


In [2]:
import yaml
import math

# Custom constructor for !degrees
def degrees_constructor(loader, node):
    value = loader.construct_scalar(node)
    return math.radians(float(value))  # Convert degrees to radians

# Register the custom constructor for the !degrees tag
yaml.add_constructor("!degrees", degrees_constructor, Loader=yaml.SafeLoader)

# Load YAML files with the custom constructor
def load_yaml(file_path):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

# Paths to the YAML files
visual_parameters = load_yaml("visual_parameters.yaml")
physical_parameters = load_yaml("physical_parameters.yaml")
joint_limits = load_yaml("joint_limits.yaml")
default_kinematics = load_yaml("default_kinematics.yaml")

# Print parsed joint limits to verify the custom constructor works
print("Parsed Joint Limits:")
print(joint_limits)

Parsed Joint Limits:
{'joint_limits': {'shoulder_pan_joint': {'has_acceleration_limits': False, 'has_effort_limits': True, 'has_position_limits': True, 'has_velocity_limits': True, 'max_effort': 54.0, 'max_position': 6.283185307179586, 'max_velocity': 3.141592653589793, 'min_position': -6.283185307179586}, 'shoulder_lift_joint': {'has_acceleration_limits': False, 'has_effort_limits': True, 'has_position_limits': True, 'has_velocity_limits': True, 'max_effort': 54.0, 'max_position': 6.283185307179586, 'max_velocity': 3.141592653589793, 'min_position': -6.283185307179586}, 'elbow_joint': {'has_acceleration_limits': False, 'has_effort_limits': True, 'has_position_limits': True, 'has_velocity_limits': True, 'max_effort': 28.0, 'max_position': 3.141592653589793, 'max_velocity': 3.141592653589793, 'min_position': -3.141592653589793}, 'wrist_1_joint': {'has_acceleration_limits': False, 'has_effort_limits': True, 'has_position_limits': True, 'has_velocity_limits': True, 'max_effort': 9.0, 'max

In [3]:
import yaml
import math

# Custom constructor for !degrees
def degrees_constructor(loader, node):
    value = loader.construct_scalar(node)
    return math.radians(float(value))  # Convert degrees to radians

# Register the custom constructor for the !degrees tag
yaml.add_constructor("!degrees", degrees_constructor, Loader=yaml.SafeLoader)

# Load YAML files with the custom constructor
def load_yaml(file_path):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

# Paths to the YAML files (update with your file paths)
visual_parameters = load_yaml("visual_parameters.yaml")
physical_parameters = load_yaml("physical_parameters.yaml")
joint_limits = load_yaml("joint_limits.yaml")
default_kinematics = load_yaml("default_kinematics.yaml")

# Start constructing the URDF
urdf_content = '<robot name="ur3e">\n'

# Add links
for link_name, mesh_data in visual_parameters["mesh_files"].items():
    urdf_content += f'  <link name="{link_name}">\n'
    urdf_content += f'    <visual>\n'
    urdf_content += f'      <geometry>\n'
    urdf_content += f'        <mesh filename="{mesh_data["visual"]["mesh"]["path"]}" />\n'
    urdf_content += f'      </geometry>\n'
    urdf_content += f'      <material name="{mesh_data["visual"]["material"]["name"]}">\n'
    urdf_content += f'        <color rgba="{mesh_data["visual"]["material"]["color"]}" />\n'
    urdf_content += f'      </material>\n'
    urdf_content += f'    </visual>\n'
    urdf_content += f'    <collision>\n'
    urdf_content += f'      <geometry>\n'
    urdf_content += f'        <mesh filename="{mesh_data["collision"]["mesh"]["path"]}" />\n'
    urdf_content += f'      </geometry>\n'
    urdf_content += f'    </collision>\n'
    urdf_content += f'  </link>\n'

# Add joints
for joint_name, joint_data in joint_limits["joint_limits"].items():
    parent_link = joint_name.split("_joint")[0]  # Derive parent link from joint name
    child_link = parent_link  # Simplified assumption (adjust as needed)
    urdf_content += f'  <joint name="{joint_name}" type="revolute">\n'
    urdf_content += f'    <parent link="{parent_link}" />\n'
    urdf_content += f'    <child link="{child_link}" />\n'

    # Handle missing keys gracefully
    min_position = joint_data.get("min_position", 0)  # Default to 0 if missing
    max_position = joint_data.get("max_position", 0)  # Default to 0 if missing
    max_velocity = joint_data.get("max_velocity", 0)  # Default to 0 if missing
    max_effort = joint_data.get("max_effort", 0)  # Default to 0 if missing

    urdf_content += f'    <limit lower="{min_position}" upper="{max_position}" velocity="{max_velocity}" effort="{max_effort}" />\n'
    urdf_content += f'  </joint>\n'

# Finish the URDF file
urdf_content += "</robot>"

# Save the URDF to a file
with open("ur3e_generated.urdf", "w") as urdf_file:
    urdf_file.write(urdf_content)

print("URDF file has been generated: ur3e_generated.urdf")



URDF file has been generated: ur3e_generated.urdf


In [ ]:
import pybullet as p
import pybullet_data

# Connect to the physics server (use GUI or DIRECT)
physics_client = p.connect(p.GUI)  # Use p.DIRECT for headless mode

# Set the search path for URDF files
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load the URDF
robot_id = p.loadURDF("ur3e_generated.urdf", [0, 0, 0], useFixedBase=True)

# Disconnect from the server when done
p.disconnect()


In [ ]:
import pybullet as p
import pybullet_data
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Connect to PyBullet in DIRECT mode (no GUI)
p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load a flat plane as the ground
plane_id = p.loadURDF("plane.urdf")

# Load the generated UR3e URDF file
urdf_path = "ur3e_generated.urdf"  # Ensure this file exists in the working directory
robot_id = p.loadURDF(urdf_path, [0, 0, 0], useFixedBase=True)

# Set gravity
p.setGravity(0, 0, -9.81)

# Camera parameters for capturing images
camera_params = {
    "width": 640,
    "height": 480,
    "view_matrix": p.computeViewMatrix(
        cameraEyePosition=[1, 1, 1],
        cameraTargetPosition=[0, 0, 0.5],
        cameraUpVector=[0, 0, 1],
    ),
    "projection_matrix": p.computeProjectionMatrixFOV(
        fov=60, aspect=1.0, nearVal=0.1, farVal=10.0
    ),
}

# Simulate the robot and capture frames
frames = []
num_frames = 100
for step in range(num_frames):
    p.stepSimulation()

    # Capture the frame
    img = p.getCameraImage(
        camera_params["width"],
        camera_params["height"],
        viewMatrix=camera_params["view_matrix"],
        projectionMatrix=camera_params["projection_matrix"],
    )
    frame = np.reshape(img[2], (camera_params["height"], camera_params["width"], 4))
    frames.append(frame)

# Disconnect from the simulation
p.disconnect()

# Create an animation
fig, ax = plt.subplots()
im = ax.imshow(frames[0])

def update(frame):
    im.set_data(frame)
    return [im]

ani = FuncAnimation(fig, update, frames=frames, interval=50, blit=True)
plt.close()

# Save the animation as a GIF
ani.save("ur3e_simulation.gif", fps=20)

# Save a single frame as an image
plt.imsave("ur3e_simulation_image.png", frames[0])

print("GIF and image have been saved as 'ur3e_simulation.gif' and 'ur3e_simulation_image.png'.")
